In [1]:
from rdflib import Graph
from rdflib.term import URIRef, Literal, BNode
from rdflib.namespace import RDF, RDFS, OWL, XSD, SKOS
import os

In [2]:
paths = {
    'conference': '/home/guilherme/Documents/kg/complex/conference/ont',
    'populated_conference': '/home/guilherme/Documents/kg/complex/conference_100/ont',
    'geolink': '/home/guilherme/Documents/kg/complex/geolink',
    'hydrography': '/home/guilherme/Documents/kg/complex/hydrography_ontology/ontology',
    'taxon': '/home/guilherme/Documents/kg/complex/taxon/ont'
}
nb = 0
for ont_name, v in paths.items():
    
    for p, d, f in os.walk(v):
        for fs in f:
            if not fs.endswith('.owl') and not fs.endswith('.rdf'):
                continue
            
                
            g = Graph().parse(os.path.join(p, fs))
            
            
            
            sbj = set(g.subjects())
            
            
            cls = set()
            ppts = set()
            
            
            for s in sbj:
                
                if type(s) == BNode:
                    continue

                tp = g.value(s, RDF.type)
                ttp = g.value(tp, RDF.type)
                
                if ttp == OWL.Class:
                    cls.add(tp)
            
            
            pred = set()
            
            spm = {}
            
            for q, w, e in g:
                
                wt = g.value(w, RDF.type)
                if wt == OWL.ObjectProperty or wt == OWL.DatatypeProperty:
                    pred.add(w)
                    
                    if w not in spm:
                        spm[w] = [set(), set()]
                        
                    spm[w][0].add(q)
                    spm[w][1].add(e)
                    
            
            pps = set()
            
            for k, v in spm.items():
                if len(v[0]) / len(v[1]) > 30 and len(v[1]) < 20:
                    for o in v[1]:
                        pps.add((k, o))
            
            
            for c in cls:
                cqn = c.split('#')[-1].lower().split("/")[-1]
                
                os.makedirs(f'/home/guilherme/Documents/kg/complex/cqas/{ont_name}/{fs.split(".")[0]}', exist_ok=True)
                with open(f'/home/guilherme/Documents/kg/complex/cqas/{ont_name}/{fs.split(".")[0]}/c-{cqn}.sparql', 'w') as f:
                    f.write(f'SELECT DISTINCT ?x WHERE {{?x a <{c}>.}}')
            
            for p in ppts:
                cqn = c.split('#')[-1].lower().split("/")[-1]
                os.makedirs(f'/home/guilherme/Documents/kg/complex/cqas/{ont_name}/{fs.split(".")[0]}', exist_ok=True)
                with open(f'/home/guilherme/Documents/kg/complex/cqas/{ont_name}/{fs.split(".")[0]}/p-{cqn}.sparql', 'w') as f:
                    f.write(f'SELECT DISTINCT ?x ?y WHERE {{?x <{p}> ?y.}}')
                    
            for k, o in pps:
                cqn = c.split('#')[-1].lower().split("/")[-1]
                os.makedirs(f'/home/guilherme/Documents/kg/complex/cqas/{ont_name}/{fs.split(".")[0]}', exist_ok=True)
                with open(f'/home/guilherme/Documents/kg/complex/cqas/{ont_name}/{fs.split(".")[0]}/pv-{cqn}.sparql', 'w') as f:
                    f.write(f'SELECT DISTINCT ?x WHERE {{?x <{k}> <{o}>.}}')
                
            
                    
                